In [ ]:
# default_exp data

# data

> Read data from netdata rest api into a pandas dataframe.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
#export
import asks
import trio
import pandas as pd

In [ ]:
#export


async def get_chart(api_call, data, col_sep='|'):
    """Get data for an individual chart.
    """
    url, chart = api_call
    r = await asks.get(url)
    r_json = r.json()
    df = pd.DataFrame(r_json['data'], columns=['time_idx'] + r_json['labels'][1:])
    df = df.set_index('time_idx').add_prefix(f'{chart}{col_sep}')
    data[chart] = df
    

async def get_charts(api_calls, col_sep='|'):
    """Create a nursey to make seperate async calls to get each chart.
    """
    data = {}
    with trio.move_on_after(60):
        async with trio.open_nursery() as nursery:
            for api_call in api_calls:
                nursery.start_soon(get_chart, api_call, data, col_sep)
    df = pd.concat(data, join='outer', axis=1, sort=True)
    df.columns = df.columns.droplevel()
    return df


def get_data(host: str = 'london.my-netdata.io', charts: list = ['system.cpu'], after: int = -60, 
             before: int = 0, col_sep: str = '|', numeric_only: bool = False) -> pd.DataFrame:
    """Define api calls to make and any post processing to be done.
    """
    api_calls = [
        (
            f'http://{host}/api/v1/data?chart={chart}&after={after}&before={before}&format=json',
            chart
        )
        for chart in charts
    ]
    df = trio.run(get_charts, api_calls, col_sep)
    if numeric_only:
        df = df._get_numeric_data()
    return df

In [ ]:
# examples

df = get_data()
print(df.shape)
df.head()

(60, 9)


,system.cpu|guest_nice,system.cpu|guest,system.cpu|steal,system.cpu|softirq,system.cpu|irq,system.cpu|user,system.cpu|system,system.cpu|nice,system.cpu|iowait
time_idx,,,,,,,,,
1592256204,0,0,0.0,0.000000,0,0.250000,1.250000,0,0.000000
1592256205,0,0,0.0,0.250627,0,0.751880,0.501253,0,0.000000
1592256206,0,0,0.0,0.000000,0,0.505050,0.505050,0,0.000000
1592256207,0,0,0.0,0.000000,0,1.250000,1.000000,0,0.000000
1592256208,0,0,0.0,0.251256,0,1.507538,0.502513,0,0.251256


In [ ]:
# tests

test_host = 'london.my-netdata.io'
test_charts = ['system.cpu', 'system.load']

df = get_data(test_host, test_charts, after=-60, before=0, col_sep='|')
# look for some expected columns
assert 'system.load|load1' in df.columns
assert 'system.cpu|user' in df.columns
# check expected shape of data
assert str(df.shape) == '(60, 12)' or '(61, 12)'
# check that all types are float64 or int64
assert len(df.dtypes[df.dtypes != 'int64'][df.dtypes != 'float64']) == 0